<a href="https://colab.research.google.com/github/kty0307/Blog/blob/main/SP_LOAN_INTRATE_DOWNLOAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **연월입력**

In [14]:
year = 2025     ## ex) 2024
month = 2       ## ex) 1

# 0. 환경설정

In [15]:
import requests
import json
import pandas as pd
import urllib.parse
import urllib3
import ssl
from urllib.parse import unquote
from requests.adapters import HTTPAdapter
from urllib3.util.ssl_ import create_urllib3_context
from google.colab import files

In [16]:
class TLSAdapter(requests.adapters.HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("AES128-SHA256")
        kwargs["ssl_context"] = ctx
        return super(TLSAdapter, self).init_poolmanager(*args, **kwargs)

In [17]:
class NewTLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("AES128-SHA256")  # 원하는 Cipher Suite 설정
        kwargs["ssl_context"] = ctx
        return super().init_poolmanager(*args, **kwargs)

In [18]:
def subtract_month(df, col_name, new_col_name):
    df[new_col_name] = pd.to_datetime(df[col_name], format='%Y%m') - pd.DateOffset(months=1)
    df[new_col_name] = df[new_col_name].dt.strftime('%Y%m').astype(int)
    return df

# 1. 저축은행

## 1.1. 신용점수별 금리

In [19]:
def savings_cs(year, month):
  url = "https://www.fsb.or.kr/ratloanconf_0200.jct"
  payload = {
      "_JSON_": json.dumps({
          "SORT_COLUMN": "",
          "SORT": "",
          "PRE_MONTH_MONEY": "",
          "SUBMIT_MONTH": f"{year:04d}{month:02d}"
      })
  }

  response = requests.post(url, data=payload)

  data = response.text

  json_data = json.loads(data)

  # 데이터 추출 및 DataFrame 생성
  df = pd.DataFrame(json_data['REC'])
  df = df[['BANK_NAME', 'SUBMIT_MONTH', 'A_RATE1_3', 'A_RATE1', 'A_RATE2',
            'A_RATE3', 'A_RATE_AVE']]

  df.columns = ['사명','공시연월','900점대','800점대','700점대','600점대','평균금리']

  df['업권'] = '저축은행업권'

  df = subtract_month(df, '공시연월', '연월')

  df = df[['사명','연월','업권','평균금리','900점대','800점대','700점대','600점대','공시연월']]

  return df

## 1.2. 금리대별 취급비중

In [20]:
def savings_int(year, month):
    url = "https://www.fsb.or.kr/ratloanconf_0300.jct"

    payload = {
        "SORT": "",
        "SUBMIT_MONTH": f"{year}{month:02}"
    }

    response = requests.post(url, data={"_JSON_": json.dumps(payload)})
    response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    df = pd.DataFrame(data["REC"])  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    desired_columns = ["BANK_NAME", "SUBMIT_MONTH", "HANDING_WEIGHT_10", "HANDING_WEIGHT_12", "HANDING_WEIGHT_14"
                        ,"HANDING_WEIGHT_16", "HANDING_WEIGHT_18", "HANDING_WEIGHT_20"]
    df = df[desired_columns] #원하는 열만 남기기

    df.columns = ['사명','제출연월','10%이하','12%이하','14%이하','16%이하','18%이하','20%이하']

    df['업권'] = '저축은행업권'

    return df

# 2. 신용카드

## 2.1. 카드론_신용점수별 금리

In [21]:
def card_loan_cs(year, month):
    url = "https://gongsi.crefia.or.kr/portal/creditcard/creditcardDisclosureDetail25Ajax"

    # cgc_seq(월)을 찾기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": 1458,
        "cgcMode": 25,
        "cgcYyyy": year,
        "mcSeq": []
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    pick_month = json.loads(response.text)  # 응답 문자열 반환

    for item in pick_month['configListMm']:
        if item['cgcquarter'] == month:
            cgc_seq = item['cgcSeq']
            break  # 첫 번째로 찾은 값만 출력하고 반복문 종료

    # 실제 데이터를 추출하기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": cgc_seq,
        "cgcMode": 25,
        "cgcYyyy": year,
        "mcSeq": [31, 96, 1, 106, 14, 13, 12, 98, 502, 108, 619, 11, 97, 105, 103, 22]
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    pick_data = data["resultList"]
    df = pd.DataFrame(pick_data)  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    df = df[["mcCompany", "cgcSeq", "cgCardPoint1", "cgCardPoint2", "cgCardPoint3"
                ,"cgCardPoint4", "cgCardPointAvg"]] #원하는 열만 남기기

    df["cgcSeq"] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    df.columns = ['사명','공시연월','900점대','800점대','700점대','600점대','평균금리']

    df['업권'] = '신용카드업권(카드론)'

    df = subtract_month(df, '공시연월', '연월')

    df = df[['사명','연월','업권','평균금리','900점대','800점대','700점대','600점대','공시연월']]

    return df

## 2.2. 현금서비스_신용점수별 금리

In [22]:
def card_cash_cs(year, month):
    url = "https://gongsi.crefia.or.kr/portal/creditcard/creditcardDisclosureDetail20Ajax"

    # cgc_seq(월)을 찾기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": 1460,
        "cgcMode": 20,
        "cgcYyyy": year,
        "mcSeq": []
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    pick_month = json.loads(response.text)  # 응답 문자열 반환

    for item in pick_month['configListMm']:
        if item['cgcquarter'] == month:
            cgc_seq = item['cgcSeq']
            break  # 첫 번째로 찾은 값만 출력하고 반복문 종료

    # 실제 데이터를 추출하기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": cgc_seq,
        "cgcMode": 20,
        "cgcYyyy": year,
        "mcSeq": [31, 96, 1, 106, 14, 13, 12, 98, 502, 108, 619, 11, 97, 105, 103, 22]
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    pick_data = data["resultList"]
    df = pd.DataFrame(pick_data)  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    df = df[["mcCompany", "cgcSeq", "cgMoneyPoint1", "cgMoneyPoint2", "cgMoneyPoint3"
                ,"cgMoneyPoint4", "cgMoneyPointAvg"]] #원하는 열만 남기기

    df["cgcSeq"] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    df.columns = ['사명','공시연월','900점대','800점대','700점대','600점대','평균금리']

    df['업권'] = '신용카드업권(현금서비스)'

    df = subtract_month(df, '공시연월', '연월')

    df = df[['사명','연월','업권','평균금리','900점대','800점대','700점대','600점대','공시연월']]

    return df

# 3. 캐피탈

## 3.1. 신용점수별 금리

In [23]:
def capital_cs(year, month):
    url = "https://gongsi.crefia.or.kr/portal/creditloan/creditloanDisclosureDetail11/ajax"

    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
          "clgcMode": 11,
          "cardItem": "134,39,40,623,130,41,25,156,6,55,32,58,52,61,57,64",
          "clgcSeq": 521,
          "clgcYyyy": 2024
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    pick_month = json.loads(response.text)  # 응답 문자열 반환

    for item in pick_month['configListMm']:
        if item['clgcquarter'] == month:
            clgc_seq = item['clgcSeq']
            break  # 첫 번째로 찾은 값만 출력하고 반복문 종료

    # 실제 데이터를 추출하기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
          "clgcMode": 11,
          "cardItem": "134,39,40,623,130,41,25,156,6,55,32,58,52,61,57,64",
          "clgcSeq": clgc_seq,
          "clgcYyyy": year
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    pick_data = data["resultList"]
    df = pd.DataFrame(pick_data)  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    df = df[["mcCompany", "clgcSeq", "clgPoint1", "clgPoint2", "clgPoint3"
                ,"clgPoint4", "clgPointAvg"]] #원하는 열만 남기기

    df["clgcSeq"] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    df.columns = ['사명','공시연월','900점대','800점대','700점대','600점대','평균금리']

    df['업권'] = '캐피탈업권'

    df = subtract_month(df, '공시연월', '연월')

    df = df[['사명','연월','업권','평균금리','900점대','800점대','700점대','600점대','공시연월']]

    return df


# 4. 은행(추후지원예정)

# 5. 데이터 처리 및 다운로드

In [24]:
file_names = f'SP_INTRATE_{year}{month:02d}.csv'

In [25]:
df_1 = savings_cs(year, month)
df_2 = card_loan_cs(year, month)
df_3 = card_cash_cs(year, month)
df_4 = capital_cs(year, month)

df = pd.concat([df_1, df_2, df_3, df_4], ignore_index=True)
df.to_csv(file_names, index=False)

In [26]:
files.download(file_names)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>